# Data Science Academy

# Projeto de Análise de Dados com Linguagem Python

## Projeto 6 - Regras de Associação e Market Basket Analysis (MBA)

### Pacotes Python Usados no Projeto

In [1]:
!pip install -q efficient_apriori

In [2]:
# Imports
import numpy as np
import pandas as pd
import efficient_apriori
import matplotlib.pyplot as plt
from datetime import datetime
from efficient_apriori import apriori
import warnings
warnings.filterwarnings('ignore')

In [3]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



### Carregando os Dados

In [4]:
# Carregando os Dados
dados_departamentos = pd.read_csv('Cap10/departamentos.csv')
dados_corredores = pd.read_csv('Cap10/corredores.csv')
dados_produtos = pd.read_csv('Cap10/produtos.csv')
dados_pedidos = pd.read_csv('Cap10/pedidos.csv')
dados_transacoes = pd.read_csv('Cap10/transacoes.csv') 


### Análise Exploratória

#### Detalhes do Tamanho de Cada Dataset via Programação

In [5]:
# Total de registros por dataset
conta_arrays = np.array([['dados_departamentos', len(dados_departamentos), dados_departamentos.shape[1]],
                       ['dados_corredores', len(dados_corredores), dados_corredores.shape[1]],
                       ['dados_produtos', len(dados_pedidos), dados_pedidos.shape[1]],
                       ['dados_transacoes', len(dados_transacoes), dados_transacoes.shape[1]]])

In [6]:
# Converte o array em dataframe
contagem = pd.DataFrame(conta_arrays, columns = ['Arquivo', 'Total de Registros', 'Total_Colunas'])

In [7]:
print(contagem)

               Arquivo Total de Registros Total_Colunas
0  dados_departamentos                 21             2
1     dados_corredores                134             2
2       dados_produtos            3421083             7
3     dados_transacoes           32434489             4


In [8]:
# Tabela de Departamentos
dados_departamentos.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [9]:
dados_corredores.shape

(134, 2)

In [10]:
# Tabela de Corredores
dados_corredores.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [11]:
# Tabela de Produtos
dados_produtos.shape

(49688, 4)

In [12]:
dados_produtos.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [13]:
dados_pedidos.shape

(3421083, 7)

In [14]:
# Tabela Pedidos
dados_pedidos.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [15]:
dados_transacoes.shape

(32434489, 4)

In [16]:
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### Verificando Valores Ausentes

In [17]:
dados_departamentos.isna().sum()

department_id    0
department       0
dtype: int64

In [18]:
dados_corredores.isna().sum()

aisle_id    0
aisle       0
dtype: int64

In [19]:
dados_produtos.isna().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64

In [20]:
dados_pedidos.isna().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [21]:
dados_transacoes.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

### *Reorganização dos dados*

#### Merge das tabelas

#### O merge só é possível quando existe um relacionamento entre os dados.

Como estratégia escolhi carregar mesclar uma tabela de cada vez.

In [23]:
%%time
dados_transacoes = dados_transacoes.merge(dados_pedidos, on = 'order_id', how = 'left')


CPU times: total: 18 s
Wall time: 19.3 s


In [25]:
%%time
dados_transacoes = dados_transacoes.merge(dados_produtos, on = 'product_id', how = 'left')
#dados_transacoes = dados_transacoes.merge(dados_corredores, on = 'aisle_id', how = 'left')
#dados_transacoes = dados_transacoes.merge(dados_departamentos, on = 'department_id', how = 'left')

CPU times: total: 25.6 s
Wall time: 30.8 s


In [27]:
%%time
#dados_transacoes = dados_transacoes.merge(dados_corredores, on = 'aisle_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_departamentos, on = 'department_id', how = 'left')

CPU times: total: 45.7 s
Wall time: 1min 3s


In [29]:
%%time
dados_transacoes = dados_transacoes.merge(dados_departamentos, on = 'department_id', how = 'left')

CPU times: total: 17.3 s
Wall time: 19 s


In [33]:
# Checando o número de colunas.
len(dados_transacoes.columns.to_list())

15

In [34]:
# Visualização da tabela completa depois do .merge()
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


In [37]:
%%time
dados_transacoes.isna().sum()

CPU times: total: 7.33 s
Wall time: 7.38 s


order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
department                      0
dtype: int64

In [36]:
%%time
dados_transacoes.isnull().sum() / len(dados_transacoes) * 100

CPU times: total: 8.09 s
Wall time: 8.26 s


order_id                  0.00000
product_id                0.00000
add_to_cart_order         0.00000
reordered                 0.00000
user_id                   0.00000
eval_set                  0.00000
order_number              0.00000
order_dow                 0.00000
order_hour_of_day         0.00000
days_since_prior_order    6.40697
product_name              0.00000
aisle_id                  0.00000
department_id             0.00000
aisle                     0.00000
department                0.00000
dtype: float64